In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = SentenceTransformer("abhinand/MedEmbed-base-v0.1", device=device)

d1 = pd.read_csv("/kaggle/input/descriptor/descriptor-terms.tsv", sep="\t")

terms = d1['TermName'].tolist()
term_ids = d1['DescriptorUI'].tolist()

Using device: cuda


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
df=pd.read_excel("/kaggle/input/descriptor/usecase_4_.xlsx")

In [ ]:
condition=pd.DataFrame(df["Conditions"].unique())
conditions=condition[0].to_list()

In [ ]:

term_embeddings = model.encode(terms, device=device)
condition_embeddings = model.encode(conditions, device=device)

Batches:   0%|          | 0/7020 [00:00<?, ?it/s]

Batches:   0%|          | 0/246 [00:00<?, ?it/s]

In [ ]:
def find_top_k_matches(condition_embedding, term_embeddings, terms, term_ids, k=3):
    similarities = np.dot(term_embeddings, condition_embedding) / (
        np.linalg.norm(term_embeddings, axis=1) * np.linalg.norm(condition_embedding)
    )

    top_k_indices = np.argsort(similarities)[-k:][::-1]

    top_k_matches = []
    for idx in top_k_indices:
        top_k_matches.append({
            "TermID": term_ids[idx],
            "TermName": terms[idx],
            "Similarity": float(similarities[idx])
        })
    return top_k_matches

In [ ]:

from tqdm import tqdm

k = 5
result = {}
for condition, condition_embedding in tqdm(zip(conditions, condition_embeddings), total=len(conditions), desc="Processing conditions"):
    top_k_matches = find_top_k_matches(condition_embedding, term_embeddings, terms, term_ids, k)
    result[condition] = top_k_matches


json_result = json.dumps(result, indent=4)

Processing conditions: 100%|██████████| 7860/7860 [30:01<00:00,  4.36it/s]


In [ ]:
output_file = "/kaggle/working/output.json"
with open(output_file, "w") as f:
    json.dump(result, f, indent=4)

print(f"JSON result saved to {output_file}")

JSON result saved to /kaggle/working/output.json
